# Visualización Interactiva SIMCE
Análisis de puntajes SIMCE por región y comuna con drill-down interactivo

A) ¿Cómo han evolucionado los puntajes promedio SIMCE (general, matemática y lenguaje) a lo largo del tiempo en cada región, y qué regiones muestran mejoras o retrocesos más marcados entre años?

B) Dentro de una misma región, ¿qué tan grande es la brecha de puntajes SIMCE entre sus comunas, y cuáles comunas se mantienen sistemáticamente por encima o por debajo del promedio regional para cada curso?

In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML
import numpy as np
import ipywidgets as widgets
from pathlib import Path
import unicodedata
import re

## Cargar Datasets

In [2]:
# Cargar datos desde carpetas con múltiples años

def cargar_curso(dir_path, curso, col_lectura, col_matematica):
    archivos = sorted(Path(dir_path).glob('*.csv'))
    if not archivos:
        raise FileNotFoundError(f"No se encontraron CSV en {dir_path}")
    dataframes = []
    for archivo in archivos:
        df_tmp = pd.read_csv(archivo)
        df_tmp['curso'] = curso
        dataframes.append(df_tmp)
    df_curso = pd.concat(dataframes, ignore_index=True)
    cols = ['agno', 'nom_reg_rbd', 'nom_com_rbd', col_lectura, col_matematica, 'curso']
    df_filtrado = df_curso[cols].copy()
    df_filtrado['promedio'] = df_filtrado[[col_lectura, col_matematica]].mean(axis=1)
    df_filtrado = df_filtrado.rename(columns={
        col_lectura: 'lectura',
        col_matematica: 'matematica'
    })
    return df_filtrado

# Directorios con CSV por curso
dir_2m = Path('datasets/2do medio')
dir_4b = Path('datasets/4tobasico')

df_2m_filtered = cargar_curso(dir_2m, '2° Medio', 'prom_lect2m_rbd', 'prom_mate2m_rbd')
df_4b_filtered = cargar_curso(dir_4b, '4° Básico', 'prom_lect4b_rbd', 'prom_mate4b_rbd')

df = pd.concat([df_2m_filtered, df_4b_filtered], ignore_index=True)
df = df.dropna(subset=['promedio', 'nom_reg_rbd', 'nom_com_rbd'])

available_years = sorted(df['agno'].unique().tolist())
print(f"Total de registros: {len(df)}")
print(f"Años disponibles: {available_years}")
print(f"Regiones: {df['nom_reg_rbd'].nunique()}")

Total de registros: 78247
Años disponibles: [2014, 2015, 2016, 2017, 2018, 2022, 2023, 2024]
Regiones: 79


### Normalización de Regiones

In [3]:
# Normalizar nombres de regiones con notación común

def _strip_accents(texto: str) -> str:
    return ''.join(
        c for c in unicodedata.normalize('NFKD', texto)
        if not unicodedata.combining(c)
    )

STOPWORDS = {"REGION", "DE", "DEL", "LA", "LAS", "LOS", "Y", "E", "EL", "GENERAL", "CARLOS", "DON", "DO", "DA"}
TOKEN_SYNONYMS = {
    "AISEN": "AYSEN",
    "HIGGINS": "LIBERTADOR",
    "OHIGGINS": "LIBERTADOR",
    "SANTIAGO": "METROPOLITANA",
    "RM": "METROPOLITANA"
}

REGION_RULES = [
    ({"ARICA", "PARINACOTA"}, "Región de Arica y Parinacota (XV)"),
    ("TARAPACA", "Región de Tarapacá (I)"),
    ("ANTOFAGASTA", "Región de Antofagasta (II)"),
    ("ATACAMA", "Región de Atacama (III)"),
    ("COQUIMBO", "Región de Coquimbo (IV)"),
    ("VALPARAISO", "Región de Valparaíso (V)"),
    ("METROPOLITANA", "Región Metropolitana de Santiago (RM)"),
    ("LIBERTADOR", "Región del Libertador General Bernardo O’Higgins (VI)"),
    ("MAULE", "Región del Maule (VII)"),
    ("NUBLE", "Región de Ñuble (XVI)"),
    ("BIOBIO", "Región del Biobío (VIII)"),
    ("ARAUCANIA", "Región de La Araucanía (IX)"),
    ("RIOS", "Región de Los Ríos (XIV)"),
    ("LAGOS", "Región de Los Lagos (X)"),
    ("AYSEN", "Región de Aysén del General Carlos Ibáñez del Campo (XI)"),
    ("MAGALLANES", "Región de Magallanes y de la Antártica Chilena (XII)")
]

# Aplicar normalización

def _tokenize_region(nombre: str) -> set:
    cleaned = re.sub(r"[^A-Z ]", " ", _strip_accents(nombre).upper())
    tokens = set()
    for token in cleaned.split():
        if token in STOPWORDS:
            continue
        token = TOKEN_SYNONYMS.get(token, token)
        tokens.add(token)
    return tokens

unmatched_regiones = set()

def normalizar_region(nombre):
    if pd.isna(nombre):
        return nombre
    original = str(nombre).strip()
    tokens = _tokenize_region(original)
    for key, canonical in REGION_RULES:
        required = key if isinstance(key, set) else {key}
        if required.issubset(tokens):
            return canonical
    unmatched_regiones.add(original)
    return original.title()

regiones_antes = df['nom_reg_rbd'].nunique()
df['nom_reg_rbd'] = df['nom_reg_rbd'].apply(normalizar_region)
print(f"Regiones únicas antes: {regiones_antes} -> después: {df['nom_reg_rbd'].nunique()}")
if unmatched_regiones:
    print("Regiones sin normalizar (revisar diccionario):")
    for reg in sorted(unmatched_regiones):
        print(f" - {reg}")

Regiones únicas antes: 79 -> después: 16


### Normalización de Comunas

In [ ]:
# Normalizar nombres de comunas
CITIES_PER_REGION = {
    "Región de Arica y Parinacota (XV)": [
        "Arica", "Camarones", "Putre", "General Lagos"
    ],
    "Región de Tarapacá (I)": [
        "Iquique", "Huara", "Colchane", "Pozo Almonte", "Alto Hospicio", 
        "Camiña", "Pica"
    ],
    "Región de Antofagasta (II)": [
        "Tocopilla", "María Elena", "Calama", "Antofagasta", "Mejillones", 
        "Taltal", "San Pedro de Atacama", "Sierra Gorda", "Ollagüe"
    ],
    "Región de Atacama (III)": [
        "Chañaral", "Diego de Almagro", "Copiapó", "Caldera", "Vallenar", 
        "Huasco", "Freirina", "Alto del Carmen", "Tierra Amarilla"
    ],
    "Región de Coquimbo (IV)": [
        "La Serena", "Coquimbo", "Andacollo", "Vicuña", "Ovalle", 
        "Combarbalá", "Illapel", "Salamanca", "Los Vilos", "Canela", 
        "Monte Patria", "Punitaqui", "Río Hurtado", "Paihuano", "La Higuera"
    ],
    "Región de Valparaíso (V)": [
        "La Ligua", "Petorca", "Cabildo", "Zapallar", "Los Andes", 
        "Calle Larga", "Rinconada", "San Felipe", "Putaendo", "Santa María", 
        "LlaiLlay", "Catemu", "Quillota", "La Cruz", "La Calera", "Nogales", 
        "Hijuelas", "Limache", "Olmué", "Valparaíso", "Viña del Mar", 
        "Concón", "Quilpué", "Quintero", "Puchuncaví", "Villa Alemana", 
        "Casablanca", "Juan Fernández", "San Antonio", "Cartagena", 
        "El Tabo", "El Quisco", "Algarrobo", "Santo Domingo", "Panquehue", 
        "Isla de Pascua", "Papudo", "San Esteban"
    ],
    "Región del Libertador General Bernardo O’Higgins (VI)": [
        "Rancagua", "Graneros", "Mostazal", "Codegua", "Machalí", 
        "Requínoa", "Rengo", "Quinta de Tilcoco", "San Vicente", 
        "Pichidegua", "Peumo", "Coltauco", "Coinco", "Doñihue", 
        "Las Cabras", "San Fernando", "Chimbarongo", "Placilla", 
        "Nancagua", "Santa Cruz", "Lolol", "Palmilla", "Peralillo", 
        "Pichilemu", "Navidad", "Litueche", "Marchihue", "Malloa", 
        "Paredones", "Olivar", "Chépica", "La Estrella", "Pumanque"
    ],
    "Región del Maule (VII)": [
        "Curicó", "Teno", "Romeral", "Molina", "Hualañé", "Licantén", 
        "Vichuquén", "Talca", "Pelarco", "Río Claro", "San Clemente", 
        "Maule", "Empedrado", "Pencahue", "Constitución", "Curepto", 
        "Linares", "Yerbas Buenas", "Colbún", "Longaví", "Parral", 
        "Retiro", "Villa Alegre", "San Javier", "Cauquenes", "Pelluhue", 
        "Chanco", "Rauco", "Sagrada Familia", "San Rafael"
    ],
    "Región del Biobío (VIII)": [
        "Chillán", "Chillán Viejo", "San Carlos", "Ñiquén", "San Fabián", 
        "Pinto", "San Ignacio", "El Carmen", "Yungay", "Pemuco", "Bulnes", 
        "Quillón", "Ránquil", "Portezuelo", "Coelemu", "Cobquecura", 
        "Quirihue", "Ninhue", "San Nicolás", "Los Ángeles", "Cabrero", 
        "Tucapel", "Antuco", "Santa Bárbara", "Quilaco", "Mulchén", 
        "Negrete", "Laja", "San Rosendo", "Yumbel", "Concepción", 
        "San Pedro de la Paz", "Chiguayante", "Hualpén", "Talcahuano", 
        "Penco", "Tomé", "Florida", "Hualqui", "Santa Juana", "Lota", 
        "Coronel", "Lebu", "Arauco", "Curanilahue", "Los Álamos", 
        "Cañete", "Contulmo", "Nacimiento", "Coihueco", "Quilleco", 
        "Alto Biobío", "Treguaco", "Tirúa"
    ],
    "Región de La Araucanía (IX)": [
        "Angol", "Renaico", "Collipulli", "Curacautín", "Ercilla", 
        "Victoria", "Traiguén", "Purén", "Temuco", "Padre Las Casas", 
        "Lautaro", "Vilcún", "Cunco", "Melipeuco", "Curarrehue", 
        "Pucón", "Villarrica", "Freire", "Pitrufquén", "Gorbea", 
        "Loncoche", "Saavedra", "Nueva Imperial", "Cholchol", 
        "Los Sauces", "Carahue", "Lumaco", "Perquenco", "Galvarino", 
        "Lonquimay", "Toltén", "Teodoro Schmidt"
    ],
    "Región de Los Ríos (XIV)": [
        "Valdivia", "Mariquina", "Lanco", "Los Lagos", "Futrono", 
        "Corral", "Máfil", "Panguipulli", "La Unión", "Paillaco", 
        "Río Bueno", "Lago Ranco"
    ],
    "Región de Los Lagos (X)": [
        "Osorno", "Puerto Varas", "San Pablo", "Puyehue", "Purranque", 
        "Río Negro", "San Juan de la Costa", "Puerto Montt", "Calbuco", 
        "Los Muermos", "Fresia", "Llanquihue", "Frutillar", "Castro", 
        "Ancud", "Quemchi", "Curaco de Vélez", "Quinchao", "Chonchi", 
        "Quellón", "Chaitén", "Hualaihué", "Futaleufú", "Palena", 
        "Puerto Octay", "Dalcahue", "Maullín", "Queilén", "Cochamó", 
        "Puqueldón"
    ],
    "Región de Aysén del General Carlos Ibáñez del Campo (XI)": [
        "Coyhaique", "Aysén", "Cisnes", "Guaitecas", "Chile Chico", 
        "Río Ibáñez", "Cochrane", "O'Higgins", "Lago Verde", "Tortel"
    ],
    "Región de Magallanes y de la Antártica Chilena (XII)": [
        "Natales", "Punta Arenas", "Porvenir", "Cabo de Hornos", 
        "Torres del Paine", "San Gregorio", "Primavera", 
        "Antártica", "Timaukel", "Río Verde", "Laguna Blanca"
    ],
    "Región Metropolitana de Santiago (RM)": [
        "Santiago", "Recoleta", "Independencia", "Quinta Normal", 
        "Estación Central", "Quilicura", "Maipú", "La Florida", 
        "Providencia", "Las Condes", "Vitacura", "Lo Barnechea", 
        "Ñuñoa", "La Reina", "Macul", "Peñalolén", "San Joaquín", 
        "San Miguel", "Pedro Aguirre Cerda", "La Granja", "La Pintana", 
        "San Ramón", "El Bosque", "Lo Espejo", "La Cisterna", 
        "Cerrillos", "Cerro Navia", "Lo Prado", "Pudahuel", "Renca", 
        "Huechuraba", "Conchalí", "Lampa", "Colina", "Tiltil", 
        "Puente Alto", "Pirque", "San José de Maipo", "San Bernardo", 
        "Buin", "Paine", "Calera de Tango", "Talagante", "Isla de Maipo", 
        "El Monte", "Peñaflor", "Padre Hurtado", "Melipilla", "Curacaví", 
        "San Pedro", "Alhué", "María Pinto"
    ],
    "Región de Ñuble (XVI)": [
        "Chillán", "Chillán Viejo", "San Carlos", "Ñiquén", "San Fabián", 
        "Pinto", "San Ignacio", "El Carmen", "Yungay", "Pemuco", "Bulnes", 
        "Quillón", "Ránquil", "Portezuelo", "Coelemu", "Cobquecura", 
        "Quirihue", "Ninhue", "San Nicolás", "Coihueco", "Treguaco"
    ]
}

KNOWN_PROBLEMS =  [
    ('Santa', 'Región del Biobío (VIII)'),
    ('Pichi', 'Región del Libertador General Bernardo O’Higgins (VI)'),
    ('Los Á', 'Región del Biobío (VIII)'),
    ('Quill', 'Región del Biobío (VIII)'),
    ('Puert', 'Región de Los Lagos (X)'),
    ('San J', 'Región Metropolitana de Santiago (RM)'),
    ('Chill', 'Región de Ñuble (XVI)')
]

CITIES_SYNONYMS = {
    'TIL TIL': 'TILTIL',
    'COIHAIQUE': 'COYHAIQUE',
    'AISEN': 'AYSEN',
    'PAIGUANO': 'PAIHUANO',
    'CALERA': 'LA CALERA'
}

def normalize_text(texto):
    if pd.isna(texto):
        return ""
    
    texto = str(texto).strip()
    texto = _strip_accents(texto).upper()
    
    return texto

def is_known_problem(comuna, region):
    if pd.isna(comuna) or pd.isna(region):
        return False
    
    comuna_str = str(comuna).strip()
    region_str = str(region).strip()
    
    return (comuna_str, region_str) in KNOWN_PROBLEMS

def apply_synonyms(comuna):
    if pd.isna(comuna):
        return comuna
    
    comuna_str = str(comuna).strip()
    comuna_norm = normalize_text(comuna_str)
    
    for variante, canonica in CITIES_SYNONYMS.items():
        if variante == comuna_norm:
            return canonica
    
    return comuna_str

def search_city(comuna_input, region_input):
    if pd.isna(comuna_input) or pd.isna(region_input):
        return comuna_input
    
    # Primero aplicar sinónimos si es necesario
    comuna_corregida = apply_synonyms(comuna_input)
    
    # Normalizar para búsqueda
    comuna_norm = normalize_text(comuna_corregida)
    region_str = str(region_input).strip()
    
    # Si la región no está en el diccionario, se devuelve la original
    if region_str not in CITIES_PER_REGION.keys():
        return comuna_corregida
    
    comunas_canonicas = CITIES_PER_REGION[region_str]
    coincidencias = []
    
    for comuna_canonica in comunas_canonicas:
        comuna_canonica_norm = normalize_text(comuna_canonica)
        
        # Coincidencia exacta
        if comuna_canonica_norm == comuna_norm:
            coincidencias.append(comuna_canonica)
        
        # Coincidencia por prefijo (el input es inicio de la comuna canónica)
        elif comuna_canonica_norm.startswith(comuna_norm):
            coincidencias.append(comuna_canonica)

    if coincidencias:
        return coincidencias[0]
    
    return comuna_corregida

def pipeline_normalizacion_comuna(df, col_comuna='nom_com_rbd', col_region='nom_reg_rbd'):
    df_original = df.copy()
    df_normalizado = df.copy()
    
    # Separar casos problemáticos
    print("\n1. SEPARANDO CASOS PROBLEMÁTICOS...")
    
    mascara_problematicos = df_normalizado.apply(
        lambda fila: is_known_problem(fila[col_comuna], fila[col_region]),
        axis=1
    )
    conteo_problematicos = mascara_problematicos.sum()
    print(f"   Casos problemáticos identificados: {conteo_problematicos}")
    
    # Separar dataframes
    df_problematicos = df_normalizado[mascara_problematicos].copy()
    df_a_normalizar = df_normalizado[~mascara_problematicos].copy()
    
    print(f"   Registros a normalizar: {len(df_a_normalizar)}")
    print(f"   Registros problemáticos: {len(df_problematicos)}")
    
    # Aplicar normalización
    print("\n2. APLICANDO NORMALIZACIÓN...")
    
    resultados = []

    for idx, fila in df_a_normalizar.iterrows():
        comuna_original = fila[col_comuna]
        region = fila[col_region]
        
        if pd.isna(comuna_original) or pd.isna(region):
            resultados.append(comuna_original)
            continue

        comuna_normalizada = search_city(
            comuna_original, 
            region
        )
        
        resultados.append(comuna_normalizada)
    
    # Aplicar resultados normalizados
    df_a_normalizar = df_a_normalizar.copy()
    df_a_normalizar[col_comuna] = resultados
    
    # Combinar resultados
    print("\n3. COMBINANDO RESULTADOS...")
    
    # Marcar datos problemáticos
    df_problematicos = df_problematicos.copy()
    df_problematicos['_normalizacion_status'] = 'PROBLEMATICO'
    df_problematicos['_comuna_original'] = df_problematicos[col_comuna]
    
    # Marcar datos normalizados
    df_a_normalizar = df_a_normalizar.copy()
    df_a_normalizar['_normalizacion_status'] = 'NORMALIZADO'
    df_a_normalizar['_comuna_original'] = df_original.loc[df_a_normalizar.index, col_comuna]
    
    df_final = pd.concat([df_a_normalizar, df_problematicos], ignore_index=True)
    
    # 8. Casos problemáticos
    print("\nCASOS PROBLEMÁTICOS:")
    if not df_problematicos.empty:
        problemas_agrupados = df_problematicos.groupby([col_region, col_comuna]).size().reset_index()
        problemas_agrupados.columns = [col_region, col_comuna, 'conteo']
        
        print(f"   Total casos distintos: {len(problemas_agrupados)}")
        for _, row in problemas_agrupados.iterrows():
            print(f"   {row[col_region][:25]:25} | {row[col_comuna]:20} ({row['conteo']} registros)")
    else:
        print("   No hay casos problemáticos pendientes")
    
    # 9. Validación final
    print("\nVALIDACIÓN FINAL:")
    comunas_unicas_final = df_final[col_comuna].nunique()
    comunas_unicas_validas = df_a_normalizar[col_comuna].nunique()
    comunas_unicas_original = df[col_comuna].nunique()
    
    print(f"   Comunas únicas originales: {comunas_unicas_original}")
    print(f"   Comunas únicas finales: {comunas_unicas_final}")
    print(f"   Comunas únicas válidas (normalizadas): {comunas_unicas_validas}")
    
    return df_final

print("Ejecutando pipeline de normalización...")
    
# Pipeline principal
df_normalizado = pipeline_normalizacion_comuna(
		df, 
		col_comuna='nom_com_rbd', 
		col_region='nom_reg_rbd'
)

# Separar los casos normalizados de los problemáticos
df_normalizados = df_normalizado[df_normalizado['_normalizacion_status'] == 'NORMALIZADO']
df_problematicos = df_normalizado[df_normalizado['_normalizacion_status'] == 'PROBLEMATICO']

Ejecutando pipeline de normalización...

1. SEPARANDO CASOS PROBLEMÁTICOS...
   Casos problemáticos identificados: 367
   Registros a normalizar: 77880
   Registros problemáticos: 367

2. APLICANDO NORMALIZACIÓN...

3. COMBINANDO RESULTADOS...

CASOS PROBLEMÁTICOS:
   Total casos distintos: 7
   Región Metropolitana de S | San J                (29 registros)
   Región de Los Lagos (X)   | Puert                (124 registros)
   Región de Ñuble (XVI)     | Chill                (75 registros)
   Región del Biobío (VIII)  | Los Á                (87 registros)
   Región del Biobío (VIII)  | Quill                (6 registros)
   Región del Biobío (VIII)  | Santa                (22 registros)
   Región del Libertador Gen | Pichi                (24 registros)

VALIDACIÓN FINAL:
   Comunas únicas originales: 976
   Comunas únicas finales: 353
   Comunas únicas válidas (normalizadas): 346


## Calcular Promedios Regionales

In [28]:
# Agrupar por región, año y curso (con media y desviación estándar para barras de error)
df_regional = df_normalizados.groupby(['agno', 'nom_reg_rbd', 'curso']).agg({
    'promedio': ['mean', 'std'],
    'lectura': ['mean', 'std'],
    'matematica': ['mean', 'std']
}).reset_index()

# Aplanar columnas MultiIndex
df_regional.columns = ['agno', 'nom_reg_rbd', 'curso', 
                        'promedio', 'promedio_std', 
                        'lectura', 'lectura_std', 
                        'matematica', 'matematica_std']

# Agrupar por comuna, año y curso (incluyendo conteo de establecimientos y desviación estándar)
df_comunal = df_normalizados.groupby(['agno', 'nom_reg_rbd', 'nom_com_rbd', 'curso']).agg({
    'promedio': ['mean', 'std', 'count'],
    'lectura': ['mean', 'std'],
    'matematica': ['mean', 'std']
}).reset_index()

# Aplanar columnas MultiIndex
df_comunal.columns = ['agno', 'nom_reg_rbd', 'nom_com_rbd', 'curso', 
                       'promedio', 'promedio_std', 'n_establecimientos', 
                       'lectura', 'lectura_std', 
                       'matematica', 'matematica_std']

print("Datos regionales preparados")
print(f"Columnas regionales: {df_regional.columns.tolist()}")
print(f"Columnas comunales: {df_comunal.columns.tolist()}")
df_regional.head()

Datos regionales preparados
Columnas regionales: ['agno', 'nom_reg_rbd', 'curso', 'promedio', 'promedio_std', 'lectura', 'lectura_std', 'matematica', 'matematica_std']
Columnas comunales: ['agno', 'nom_reg_rbd', 'nom_com_rbd', 'curso', 'promedio', 'promedio_std', 'n_establecimientos', 'lectura', 'lectura_std', 'matematica', 'matematica_std']


,agno,nom_reg_rbd,curso,promedio,promedio_std,lectura,lectura_std,matematica,matematica_std
0,2014,Región Metropolitana de Santiago (RM),2° Medio,260.034962,41.091799,252.372605,33.353018,267.697318,50.556466
1,2014,Región Metropolitana de Santiago (RM),4° Básico,254.812919,26.719393,258.706935,25.865418,250.949077,29.061914
2,2014,Región de Antofagasta (II),2° Medio,266.294521,40.260681,260.178082,31.315750,272.410959,50.802405
3,2014,Región de Antofagasta (II),4° Básico,254.980769,23.976846,258.884615,23.656664,251.372093,25.502166
4,2014,Región de Arica y Parinacota (XV),2° Medio,250.151515,42.579977,245.181818,34.849188,255.121212,53.029919


## Visualización Interactiva con Drill-Down

In [29]:
# Paleta de colores por macrozona (amigable con daltonismo - basada en ColorBrewer)
# Norte Grande, Norte Chico, Central, Sur, Austral
MACROZONA_COLORES = {
    # Norte Grande (tonos naranjas/rojos)
    "Región de Arica y Parinacota (XV)": "#d73027",
    "Región de Tarapacá (I)": "#f46d43",
    "Región de Antofagasta (II)": "#fdae61",
    # Norte Chico (tonos amarillos)
    "Región de Atacama (III)": "#fee090",
    "Región de Coquimbo (IV)": "#ffffbf",
    # Zona Central (tonos verdes)
    "Región de Valparaíso (V)": "#a6d96a",
    "Región Metropolitana de Santiago (RM)": "#66bd63",
    "Región del Libertador General Bernardo O'Higgins (VI)": "#1a9850",
    # Zona Sur (tonos azules)
    "Región del Maule (VII)": "#74add1",
    "Región de Ñuble (XVI)": "#4575b4",
    "Región del Biobío (VIII)": "#313695",
    "Región de La Araucanía (IX)": "#5e4fa2",
    "Región de Los Ríos (XIV)": "#8073ac",
    "Región de Los Lagos (X)": "#b2abd2",
    # Zona Austral (tonos morados/grises)
    "Región de Aysén del General Carlos Ibáñez del Campo (XI)": "#762a83",
    "Región de Magallanes y de la Antártica Chilena (XII)": "#9970ab",
}

class VisualizacionSimce:
    def __init__(self, df_regional, df_comunal):
        self.df_regional = df_regional
        self.df_comunal = df_comunal
        self.current_region = None
        self.current_curso = '2° Medio'
        self.metric_labels = {
            'promedio': 'Promedio General',
            'matematica': 'Promedio Matemática',
            'lectura': 'Promedio Lenguaje'
        }
        self.current_metric = 'promedio'
        self.fig = go.FigureWidget()
        self.btn_volver = widgets.Button(
            description='← Volver',
            icon='arrow-left',
            button_style='info',
            layout=widgets.Layout(width='140px', display='none')
        )
        self.btn_volver.on_click(self._handle_back_button)
        self.selector_curso = widgets.ToggleButtons(
            options=['2° Medio', '4° Básico'],
            value='2° Medio',
            description='Curso:',
            button_style='primary',
            layout=widgets.Layout(width='320px')
        )
        self.selector_curso.observe(self._handle_curso_change, names='value')
        self.selector_metric = widgets.ToggleButtons(
            options=[
                ('Promedio General', 'promedio'),
                ('Promedio Matemática', 'matematica'),
                ('Promedio Lenguaje', 'lectura')
            ],
            value='promedio',
            description='Indicador:',
            button_style='',
            layout=widgets.Layout(width='480px')
        )
        self.selector_metric.observe(self._handle_metric_change, names='value')
        self.title_widget = widgets.HTML(
            "<h3 style='margin:0 0 8px 0;'>Visualización SIMCE Interactiva</h3>"
        )
        self.controls = widgets.VBox([
            self.title_widget,
            widgets.HBox([
                self.selector_curso,
                widgets.Box([self.btn_volver], layout=widgets.Layout(padding='0 0 0 16px'))
            ]),
            self.selector_metric
        ])
        self.container = widgets.VBox([self.controls, self.fig])
        self.year_ticks = sorted(self.df_regional['agno'].unique().tolist())
        self.year_labels = [str(y) for y in self.year_ticks]
        self.x_padding = 7.5
        self.x_zoom_padding = 3
        self.metric_ranges = {
            col: [
                float(self.df_regional[col].min()) - self.x_zoom_padding,
                float(self.df_regional[col].max()) + self.x_zoom_padding
            ]
            for col in ['promedio', 'matematica', 'lectura']
        }
        self.y_range = [self.year_ticks[0] - 0.5, self.year_ticks[-1] + 0.5]
        # Mapeo de índices de líneas de trayectoria y barras de error por región
        self.region_line_indices = {}
        self.region_error_indices = {}
        # Set de regiones con línea fijada (seleccionadas desde leyenda)
        self.pinned_regions = set()
        # Estado previo de visibilidad para detectar cambios
        self._prev_visibility = {}
    
    def crear_grafico_regional(self, curso='2° Medio'):
        """Mostrar vista regional con líneas de trayectoria (ocultas por defecto, visibles al hover/seleccionar)"""
        self.current_curso = curso
        if self.selector_curso.value != curso:
            self.selector_curso.value = curso
        self.current_region = None
        
        # Limpiar handlers antiguos antes de recrear el gráfico
        self._cleanup_handlers()
        
        df_curso = self.df_regional[self.df_regional['curso'] == curso]
        metric_col = self.current_metric
        metric_label = self.metric_labels[metric_col]
        metric_std_col = f'{metric_col}_std'
        self.fig = go.FigureWidget()
        self.region_line_indices = {}
        self.region_error_indices = {}
        self.pinned_regions = set()
        self._prev_visibility = {}
        
        trace_idx = 0
        for region in df_curso['nom_reg_rbd'].unique():
            df_reg = df_curso[df_curso['nom_reg_rbd'] == region].sort_values('agno')
            color = MACROZONA_COLORES.get(region, '#888888')
            
            # Línea de trayectoria (oculta por defecto, punteada y delgada)
            self.fig.add_trace(go.Scatter(
                x=df_reg[metric_col],
                y=df_reg['agno'],
                mode='lines',
                name=f'{region} (trayectoria)',
                line=dict(color=color, width=1.5, dash='dot'),
                opacity=0.5,
                hoverinfo='skip',
                showlegend=False,
                visible=False,  # Oculta por defecto
                legendgroup=region
            ))
            self.region_line_indices[region] = trace_idx
            trace_idx += 1
            
            # Barras de error (ocultas por defecto)
            self.fig.add_trace(go.Scatter(
                x=df_reg[metric_col],
                y=df_reg['agno'],
                mode='markers',
                name=f'{region} (error)',
                marker=dict(size=0.1, opacity=0),  # Marcador invisible
                error_x=dict(
                    type='data',
                    array=df_reg[metric_std_col],
                    visible=True,
                    color=color,
                    thickness=1.5,
                    width=4
                ),
                hoverinfo='skip',
                showlegend=False,
                visible=False,  # Oculta por defecto
                legendgroup=region
            ))
            self.region_error_indices[region] = trace_idx
            trace_idx += 1
            
            # Puntos principales (siempre visibles por defecto)
            self.fig.add_trace(go.Scatter(
                x=df_reg[metric_col],
                y=df_reg['agno'],
                mode='markers',
                name=region,
                marker=dict(size=12, opacity=0.85, color=color, line=dict(width=1, color='white')),
                hovertemplate='<b>%{fullData.name}</b><br>' +
                              'Año: %{y}<br>' +
                              f'{metric_label}: %{{x:.1f}}<br>' +
                              '<extra></extra>',
                legendgroup=region
            ))
            self._prev_visibility[region] = True  # Inicialmente todos visibles
            trace_idx += 1
        
        self.fig.update_layout(
            title=dict(
                text=f'{metric_label} SIMCE {curso} por Región (Click para ver comunas)',
                y=0.97,
                yanchor='top',
                pad=dict(b=15)
            ),
            xaxis=dict(
                title=metric_label,
                range=self.metric_ranges[metric_col],
                autorange=False
            ),
            yaxis=dict(
                title='Año',
                tickmode='array',
                tickvals=self.year_ticks,
                ticktext=self.year_labels,
                range=self.y_range,
                autorange=False
            ),
            hovermode='closest',
            height=650,
            width=1200,
            margin=dict(l=80, r=40, b=120, t=160),
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='top',
                y=-0.15,
                xanchor='center',
                x=0.5,
                itemclick='toggleothers',  # Click = mostrar solo esa región
                itemdoubleclick='toggle'   # Doble click = toggle individual
            ),
            dragmode='pan'
        )
        
        # Observar cambios en la visibilidad de los traces
        for i, trace in enumerate(self.fig.data):
            if '(trayectoria)' not in trace.name and '(error)' not in trace.name:
                trace.on_change(self._on_trace_visibility_change, 'visible')
        
        self._attach_trace_handlers(self._on_region_click)
        self._attach_hover_handlers()
        self._set_button_visibility(False)
        self._refresh_container()
        return self.container
    
    def _attach_hover_handlers(self):
        """Conectar eventos de hover SOLO para mostrar líneas de trayectoria"""
        for trace in self.fig.data:
            if '(trayectoria)' not in trace.name and '(error)' not in trace.name:
                trace.on_hover(self._on_region_hover)
                trace.on_unhover(self._on_region_unhover)
    
    def _on_region_hover(self, trace, points, state):
        """Mostrar SOLO línea de trayectoria al pasar el mouse"""
        if points.point_inds:
            region = trace.name
            if region in self.region_line_indices:
                line_idx = self.region_line_indices[region]
                if line_idx < len(self.fig.data):
                    with self.fig.batch_update():
                        self.fig.data[line_idx].visible = True
    
    def _on_region_unhover(self, trace, points, state):
        """Ocultar línea de trayectoria al salir del hover (solo si no está fijada)"""
        region = trace.name
        if region in self.region_line_indices and region not in self.pinned_regions:
            line_idx = self.region_line_indices[region]
            if line_idx < len(self.fig.data):
                with self.fig.batch_update():
                    self.fig.data[line_idx].visible = False
    
    def _on_trace_visibility_change(self, trace, visible):
        """Detectar cuando cambia la visibilidad de un trace (por click en leyenda)"""
        region = trace.name
        if region not in self.region_line_indices:
            return
        
        line_idx = self.region_line_indices[region]
        error_idx = self.region_error_indices.get(region)
        
        # Verificar que los índices siguen siendo válidos
        if line_idx >= len(self.fig.data):
            return
        
        # Determinar si el trace está visible o no
        is_visible = visible is True or visible is None
        was_visible = self._prev_visibility.get(region, True)
        
        # Contar cuántas regiones están visibles ahora
        visible_count = 0
        for r, lidx in self.region_line_indices.items():
            # Los puntos están 2 posiciones después (línea, error, puntos)
            points_idx = lidx + 2
            if points_idx < len(self.fig.data):
                pt = self.fig.data[points_idx]
                if pt.visible is True or pt.visible is None:
                    visible_count += 1
        
        # Si solo una región está visible, fijar su línea y barras de error
        if visible_count == 1 and is_visible:
            self.pinned_regions = {region}
            with self.fig.batch_update():
                # Mostrar línea y barras de error solo de la región seleccionada
                for r, lidx in self.region_line_indices.items():
                    if lidx < len(self.fig.data):
                        self.fig.data[lidx].visible = (r == region)
                    eidx = self.region_error_indices.get(r)
                    if eidx and eidx < len(self.fig.data):
                        self.fig.data[eidx].visible = (r == region)
        elif visible_count > 1:
            # Si hay más de una visible, ocultar todas las líneas y barras de error
            self.pinned_regions = set()
            with self.fig.batch_update():
                for r, lidx in self.region_line_indices.items():
                    if lidx < len(self.fig.data):
                        self.fig.data[lidx].visible = False
                    eidx = self.region_error_indices.get(r)
                    if eidx and eidx < len(self.fig.data):
                        self.fig.data[eidx].visible = False
        
        # Actualizar estado previo
        self._prev_visibility[region] = is_visible
    
    def _on_region_click(self, trace, points, selector):
        if points.point_inds and '(trayectoria)' not in trace.name and '(error)' not in trace.name:
            region = trace.name
            self.crear_grafico_comunal(region, self.current_curso)
    
    def crear_grafico_comunal(self, region, curso):
        """Mostrar vista comunal con líneas de trayectoria, barras de error y leyenda horizontal"""
        self.current_region = region
        
        # Limpiar handlers antiguos antes de recrear el gráfico
        self._cleanup_handlers()
        
        df_curso = self.df_comunal[
            (self.df_comunal['nom_reg_rbd'] == region) &
            (self.df_comunal['curso'] == curso)
        ]
        metric_col = self.current_metric
        metric_label = self.metric_labels[metric_col]
        metric_std_col = f'{metric_col}_std'
        self.fig = go.FigureWidget()
        
        # Reiniciar índices para comunas
        self.comuna_line_indices = {}
        self.comuna_error_indices = {}
        self.pinned_comunas = set()
        self._prev_comuna_visibility = {}
        
        if df_curso.empty:
            self.fig.add_annotation(text='Sin datos para esta región/curso', showarrow=False)
            tickvals = self.year_ticks
            ticktext = self.year_labels
            x_range = self.metric_ranges[metric_col]
        else:
            # Calcular promedio y mediana por año para esta región
            stats_por_anio = df_curso.groupby('agno')[metric_col].agg(['mean', 'median']).reset_index()
            
            # Color de la región (macrozona)
            region_color = MACROZONA_COLORES.get(region, '#888888')
            
            # Crear traces por cada comuna (similar a regiones)
            comunas_unicas = sorted(df_curso['nom_com_rbd'].unique())
            
            # Generar colores para comunas usando una escala de colores
            colors = px.colors.qualitative.Plotly + px.colors.qualitative.Set2 + px.colors.qualitative.Pastel
            
            trace_idx = 0
            for idx, comuna in enumerate(comunas_unicas):
                df_comuna = df_curso[df_curso['nom_com_rbd'] == comuna].sort_values('agno')
                comuna_color = colors[idx % len(colors)]
                
                # Línea de trayectoria (oculta por defecto)
                self.fig.add_trace(go.Scatter(
                    x=df_comuna[metric_col],
                    y=df_comuna['agno'],
                    mode='lines',
                    name=f'{comuna} (trayectoria)',
                    line=dict(color=comuna_color, width=1.5, dash='dot'),
                    opacity=0.5,
                    hoverinfo='skip',
                    showlegend=False,
                    visible=False,
                    legendgroup=comuna
                ))
                self.comuna_line_indices[comuna] = trace_idx
                trace_idx += 1
                
                # Barras de error (ocultas por defecto)
                self.fig.add_trace(go.Scatter(
                    x=df_comuna[metric_col],
                    y=df_comuna['agno'],
                    mode='markers',
                    name=f'{comuna} (error)',
                    marker=dict(size=0.1, opacity=0),
                    error_x=dict(
                        type='data',
                        array=df_comuna[metric_std_col].fillna(0),
                        visible=True,
                        color=comuna_color,
                        thickness=1.5,
                        width=4
                    ),
                    hoverinfo='skip',
                    showlegend=False,
                    visible=False,
                    legendgroup=comuna
                ))
                self.comuna_error_indices[comuna] = trace_idx
                trace_idx += 1
                
                # Puntos de la comuna
                self.fig.add_trace(go.Scattergl(
                    x=df_comuna[metric_col],
                    y=df_comuna['agno'],
                    mode='markers',
                    marker=dict(
                        size=10,
                        opacity=0.7,
                        color=comuna_color
                    ),
                    customdata=df_comuna['n_establecimientos'],
                    hovertemplate='<b>' + comuna + '</b><br>' +
                                  'Año: %{y}<br>' +
                                  f'{metric_label}: %{{x:.1f}}<br>' +
                                  'Establecimientos: %{customdata}<br>' +
                                  '<extra></extra>',
                    name=comuna,
                    legendgroup=comuna
                ))
                self._prev_comuna_visibility[comuna] = True
                trace_idx += 1
            
            # Línea de promedio regional por año
            self.fig.add_trace(go.Scatter(
                x=stats_por_anio['mean'],
                y=stats_por_anio['agno'],
                mode='lines+markers',
                name='Promedio Regional',
                line=dict(color=region_color, width=3),
                marker=dict(size=10, symbol='diamond', color=region_color, line=dict(width=1, color='white')),
                hovertemplate='<b>Promedio Regional</b><br>' +
                              'Año: %{y}<br>' +
                              f'{metric_label}: %{{x:.1f}}<br>' +
                              '<extra></extra>',
                legendgroup='stats'
            ))
            
            # Línea de mediana regional por año
            self.fig.add_trace(go.Scatter(
                x=stats_por_anio['median'],
                y=stats_por_anio['agno'],
                mode='lines+markers',
                name='Mediana Regional',
                line=dict(color='#333333', width=2, dash='dash'),
                marker=dict(size=8, symbol='square', color='#333333', line=dict(width=1, color='white')),
                hovertemplate='<b>Mediana Regional</b><br>' +
                              'Año: %{y}<br>' +
                              f'{metric_label}: %{{x:.1f}}<br>' +
                              '<extra></extra>',
                legendgroup='stats'
            ))
            
            region_years = sorted(df_curso['agno'].unique().tolist())
            tickvals = region_years if region_years else self.year_ticks
            ticktext = [str(y) for y in tickvals]
            x_range = [
                float(df_curso[metric_col].min()) - self.x_padding,
                float(df_curso[metric_col].max()) + self.x_padding
            ]
        
        self.fig.update_layout(
            title=dict(
                text=f'Comunas de {region} - {curso} ({metric_label})',
                y=0.97,
                yanchor='top',
                pad=dict(b=10)
            ),
            xaxis=dict(
                title=metric_label,
                range=x_range,
                autorange=False
            ),
            yaxis=dict(
                title='Año',
                tickmode='array',
                tickvals=tickvals,
                ticktext=ticktext,
                range=[tickvals[0] - 0.5, tickvals[-1] + 0.5] if tickvals else self.y_range,
                autorange=False
            ),
            height=650,
            width=1200,
            margin=dict(l=80, r=40, b=140, t=150),
            showlegend=True,
            legend=dict(
                orientation='h',
                yanchor='top',
                y=-0.18,
                xanchor='center',
                x=0.5,
                itemclick='toggleothers',
                itemdoubleclick='toggle'
            ),
            dragmode='pan'
        )
        
        # Observar cambios en visibilidad de comunas
        for i, trace in enumerate(self.fig.data):
            if '(trayectoria)' not in trace.name and '(error)' not in trace.name and trace.name not in ['Promedio Regional', 'Mediana Regional']:
                trace.on_change(self._on_comuna_visibility_change, 'visible')
        
        self._attach_trace_handlers(self._on_back_click)
        self._attach_comuna_hover_handlers()
        self._set_button_visibility(True)
        self._refresh_container()
        return self.container
    
    def _attach_comuna_hover_handlers(self):
        """Conectar eventos de hover para líneas de trayectoria de comunas"""
        for trace in self.fig.data:
            if '(trayectoria)' not in trace.name and '(error)' not in trace.name and trace.name not in ['Promedio Regional', 'Mediana Regional']:
                trace.on_hover(self._on_comuna_hover)
                trace.on_unhover(self._on_comuna_unhover)
    
    def _on_comuna_hover(self, trace, points, state):
        """Mostrar línea de trayectoria al hover sobre comuna"""
        if points.point_inds:
            comuna = trace.name
            if comuna in self.comuna_line_indices:
                line_idx = self.comuna_line_indices[comuna]
                if line_idx < len(self.fig.data):
                    with self.fig.batch_update():
                        self.fig.data[line_idx].visible = True
    
    def _on_comuna_unhover(self, trace, points, state):
        """Ocultar línea de trayectoria al salir del hover (si no está fijada)"""
        comuna = trace.name
        if comuna in self.comuna_line_indices and comuna not in self.pinned_comunas:
            line_idx = self.comuna_line_indices[comuna]
            if line_idx < len(self.fig.data):
                with self.fig.batch_update():
                    self.fig.data[line_idx].visible = False
    
    def _on_comuna_visibility_change(self, trace, visible):
        """Detectar cambios de visibilidad en comunas por click en leyenda"""
        comuna = trace.name
        if comuna not in self.comuna_line_indices:
            return
        
        line_idx = self.comuna_line_indices[comuna]
        error_idx = self.comuna_error_indices.get(comuna)
        
        if line_idx >= len(self.fig.data):
            return
        
        is_visible = visible is True or visible is None
        
        # Contar comunas visibles
        visible_count = 0
        for c, lidx in self.comuna_line_indices.items():
            points_idx = lidx + 2  # línea, error, puntos
            if points_idx < len(self.fig.data):
                pt = self.fig.data[points_idx]
                if pt.visible is True or pt.visible is None:
                    visible_count += 1
        
        # Si solo una comuna visible, mostrar su línea y barras de error
        if visible_count == 1 and is_visible:
            self.pinned_comunas = {comuna}
            with self.fig.batch_update():
                for c, lidx in self.comuna_line_indices.items():
                    if lidx < len(self.fig.data):
                        self.fig.data[lidx].visible = (c == comuna)
                    eidx = self.comuna_error_indices.get(c)
                    if eidx and eidx < len(self.fig.data):
                        self.fig.data[eidx].visible = (c == comuna)
        elif visible_count > 1:
            # Ocultar todas las líneas y barras de error
            self.pinned_comunas = set()
            with self.fig.batch_update():
                for c, lidx in self.comuna_line_indices.items():
                    if lidx < len(self.fig.data):
                        self.fig.data[lidx].visible = False
                    eidx = self.comuna_error_indices.get(c)
                    if eidx and eidx < len(self.fig.data):
                        self.fig.data[eidx].visible = False
        
        self._prev_comuna_visibility[comuna] = is_visible
    
    def _on_back_click(self, trace, points, selector):
        self.crear_grafico_regional(self.current_curso)
    
    def _handle_back_button(self, _):
        self.crear_grafico_regional(self.current_curso)
    
    def _handle_curso_change(self, change):
        nuevo_curso = change['new']
        if nuevo_curso != self.current_curso:
            self.crear_grafico_regional(nuevo_curso)
    
    def _handle_metric_change(self, change):
        nuevo_metric = change['new']
        if nuevo_metric != self.current_metric:
            self.current_metric = nuevo_metric
            if self.current_region:
                self.crear_grafico_comunal(self.current_region, self.current_curso)
            else:
                self.crear_grafico_regional(self.current_curso)
    
    def _attach_trace_handlers(self, handler):
        for trace in self.fig.data:
            trace.on_click(handler)
    
    def _set_button_visibility(self, visible):
        self.btn_volver.layout.display = 'inline-flex' if visible else 'none'
        self.btn_volver.disabled = not visible
    
    def _refresh_container(self):
        # Forzar actualización del contenedor con el nuevo FigureWidget
        self.container.children = [self.controls, self.fig]
    
    def _cleanup_handlers(self):
        """Limpiar todos los event handlers del gráfico anterior para evitar parpadeos"""
        try:
            if hasattr(self, 'fig') and self.fig.data:
                for trace in self.fig.data:
                    # Intentar limpiar handlers si existen
                    if hasattr(trace, '_hover_callbacks'):
                        trace._hover_callbacks.clear()
                    if hasattr(trace, '_unhover_callbacks'):
                        trace._unhover_callbacks.clear()
                    if hasattr(trace, '_click_callbacks'):
                        trace._click_callbacks.clear()
        except:
            pass  # Ignorar errores si no hay handlers
    
# Crear instancia
viz = VisualizacionSimce(df_regional, df_comunal)
print("Visualización creada. Ejecuta la siguiente celda para mostrar el gráfico.")

Visualización creada. Ejecuta la siguiente celda para mostrar el gráfico.


## Mostrar Gráfico Interactivo

**Instrucciones:**
- Click en cualquier punto para ver las comunas de esa región
- Click en "← Volver" o en cualquier punto de comunas para regresar a regiones
- Puedes cambiar el curso modificando el parámetro en `crear_grafico_regional()`

In [30]:
# Mostrar gráfico para 2° Medio
viz.crear_grafico_regional('2° Medio')